In [2]:
# imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

# need to import Chroma AFTER sqlite swap
from langchain_community.vectorstores import Chroma

# Load OpenAI api key (remove before commit)
import os
import json

In [ ]:
# initial setup
os.environ["OPENAI_API_KEY"] = ""
# Load dos modelos (Embedding model da OpenAI e a LLM em si)
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens= 200)

In [ ]:
def getRelevantChunksRetriever():
    """Load PDF, split into chunks, save them to the vectordb, and return relevant chunks retriever"""
    pdf_link = "os-sertoes.pdf"
    loader = PyPDFLoader(pdf_link, extract_images=False)
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 4000, # each chunk will have 4000 characters
        chunk_overlap = 20, # they will have an overlap of 20 characters so we do not lose much content at the end of a given chunk
        length_function = len,
        add_start_index = True
    )
    
    # split PDF into chunks
    chunks = text_splitter.split_documents(pages)

    # persist chunks into the VectorDB using OPENAI embedding model to the simple_rag_db dir
    # doing in batches to avoid exceeding token limit
    #Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="simple_rag_db")
    max_batch_size = 100
    for i in range(0, len(chunks), max_batch_size):
        batch = chunks[i:i + max_batch_size]
        Chroma.from_documents(batch, embedding=embeddings_model, persist_directory="simple_rag_db")
    
    # load vectordb
    vectordb = Chroma(embedding_function=embeddings_model, persist_directory="simple_rag_db")

    # Load Retriever - function to retrieve relevant chunks
    # k = 3 - top 3 relevant chunks
    retriever = vectordb.as_retriever(search_kwargs={"k": 3}, max_tokens_limit=10000)
    return retriever

def getRelevantChunks(question):
    retriever = getRelevantChunksRetriever()
    context = retriever.invoke(question)
    return context

/tmp/ipykernel_16243/1648333908.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(embedding_function=embeddings_model, persist_directory="simple_rag_db")


VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f3fde9a7d00>, search_kwargs={'k': 3})

In [6]:
def ask(question, llm):
    TEMPLATE = """
        Você é um especialista nas obras de Euclides da cunha, em especial, na obra "Os sertões". 
        Com base no contexto, responda a pergunta indicada.
        Contexto: {context}
        Pergunta: {question}
    """
    prompt = PromptTemplate(input_variables = ['context', 'question'], template=TEMPLATE)
    sequence = RunnableSequence(prompt | llm)
    context = getRelevantChunks(question)
    response = sequence.invoke({ 'context': context, 'question': question })
    return response

In [ ]:
response = ask("1. Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?", llm)
response


AIMessage(content='Euclides da Cunha descreve o ambiente natural do sertão nordestino como extremamente adverso, com características geológicas, topográficas e climáticas que se influenciam mutuamente. Ele ressalta que as condições genéticas dos habitantes reagem fortemente a essas influências ambientais, contribuindo para agravar as condições de vida no local. Euclides da Cunha destaca a significação mesológica do sertão, apontando para um conflito perene e indefinido que afeta a vida dos habitantes, tornando-o um lugar de extrema dificuldade para sobreviver.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 2184, 'total_tokens': 2331, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': '

In [9]:
response2 = ask("2. Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?", llm)
response2

AIMessage(content='Euclides da Cunha descreve a população sertaneja como resistente, sofrida e adaptada às condições adversas do ambiente em que vivem. Ele destaca a força e a determinação dessas pessoas, que lutam diariamente contra as agruras do clima, da geografia e das condições de vida precárias. Euclides relaciona essas características com o ambiente sertanejo, mostrando como a dureza do meio influencia o modo de vida e as características do povo que ali habita. Ele destaca a interação entre as condições genéticas, as influências do ambiente e as características físicas e geológicas da região, mostrando como todos esses elementos se entrelaçam e moldam a vida e a cultura sertaneja.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 180, 'prompt_tokens': 2078, 'total_tokens': 2258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_deta

In [10]:
response3 = ask("3. Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?", llm)
response3

AIMessage(content='Segundo Euclides da Cunha, o contexto histórico e político que levou à Guerra de Canudos foi marcado pela presença de tensões entre monarquistas e republicanos, assim como pela atuação de figuras sanguinárias, como o Coronel Antônio Moreira César, que serviam aos projetos políticos do governo da época. Além disso, a derrota das três primeiras expedições militares contra o povo de Canudos contribuiu para intensificar o conflito na região. Euclides da Cunha também estabeleceu um paralelo entre a rebelião em Canudos e a revolta dos camponeses monarquistas e católicos da região da Vendeia na França, ocorrida entre 1793 e 1796.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 185, 'prompt_tokens': 2635, 'total_tokens': 2820, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},

In [11]:
response4 = ask("4. Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?", llm)
response4

AIMessage(content='Euclides da Cunha descreve a figura de Antônio Conselheiro como um líder carismático e religioso, que exercia grande influência sobre seus seguidores em Canudos. Ele é retratado como um protetor dos mais necessitados, como mulheres, crianças, velhos e doentes, que viviam sob sua tutela no arraial. Conselheiro é visto como um líder que exigia disciplina e ordem entre seus seguidores, sendo tolerante com as expedições políticas que ocorriam na região, mas mantendo seu foco na proteção dos mais vulneráveis. Ele é descrito como um Santo protetor, cuja presença era reverenciada pelos habitantes de Canudos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 164, 'prompt_tokens': 2337, 'total_tokens': 2501, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.

In [12]:
response5 = ask('5. Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?', llm)
response5

AIMessage(content='Os principais aspectos da crítica social e política presentes em "Os Sertões" estão relacionados à denúncia da injustiça social e da desigualdade presentes no interior do Brasil, especialmente no sertão nordestino. Euclides da Cunha mostra a realidade brutal e violenta dos conflitos entre os sertanejos e as forças do governo, além de abordar questões como a exploração econômica, o descaso das autoridades e a falta de infraestrutura nas regiões mais remotas do país.\n\nEsses aspectos refletem a visão do autor sobre o Brasil da época como um país marcado pela violência, pela desigualdade social e pela falta de desenvolvimento. Euclides da Cunha critica a forma como o governo lida com as questões do interior, evidenciando a falta de políticas públicas adequ', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 200, 'prompt_tokens': 1363, 'total_tokens': 1563, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_t